<a href="https://colab.research.google.com/github/glee1228/netVLad_keras/blob/master/test_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf

from torch.autograd import Variable
from torchvision import models

from tensorflow import keras
from keras.preprocessing import image

import PIL
import torch
import torchvision

import matplotlib.pyplot as plt
from sklearn import svm, datasets, metrics

Using TensorFlow backend.


In [2]:
torch.manual_seed(777)

In [0]:
# !pip install tensorboardcolab
# from tensorboardcolab import TensorBoardColab
# tb = TensorBoardColab()

In [4]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
    drive.mount('/content/gdrive')
    print('Google Drive is mounted\n')
else:
    print('Google Drive is already mounted\n')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Google Drive is mounted



In [0]:
ckpt_path = '/content/gdrive/My Drive/AILeader_Dataset/ckpt'
test_path = '/content/gdrive/My Drive/AILeader_Dataset/test'
history_path = '/content/gdrive/My Drive/AILeader_Dataset/history_train.csv'

epochs = 50
batch_size = 256
model_name = 'resnet18' #'resnet18','resnet50','resnet101','alexnet','vgg16','vgg19','densenet121','densenet161','mobilenet','squeezenet'

In [0]:
model_save_name = '{}_model_{}_{}.pt'.format(model_name,epochs-1,batch_size)
# model_save_name = 'model_49.pt'
model_load_path = os.path.join(ckpt_path,model_save_name)

In [7]:
refer_num_Images = 0
for path, dirs, files in os.walk(test_path):
  for directory in dirs:
    files_len=len(os.listdir(os.path.join(path,directory)))
    print(files_len)
    refer_num_Images+=files_len
print('Total Number of Images(dataset)  : {}'.format(refer_num_Images))

24
7
14
15
23
Total Number of Images(dataset)  : 83


In [0]:
## OOM으로 인한 쿼리 이미지 수 축소
## train 데이터셋(366장)으로 test할 경우  refer_num_Images != query_num_Images
## test 데이터셋(81장)으로 test 할 경우 refer_num_Images == query_num_Images
query_num_Images = 100

In [0]:
class NetVLAD(nn.Module):
    """NetVLAD layer implementation"""

    def __init__(self, num_clusters=7, dim=128, alpha=100.0,
                 normalize_input=True):
        """
        Args:
            num_clusters : int
                The number of clusters
            dim : int
                Dimension of descriptors
            alpha : float
                Parameter of initialization. Larger value is harder assignment.
            normalize_input : bool
                If true, descriptor-wise L2 normalization is applied to input.
        """
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = alpha
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))
        self._init_params()

    def _init_params(self):
        self.conv.weight = nn.Parameter(
            (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
        )
        self.conv.bias = nn.Parameter(
            - self.alpha * self.centroids.norm(dim=1)
        )

    def forward(self, x):
        N, C = x.shape[:2]

        if self.normalize_input:
            x = F.normalize(x, p=2, dim=1)  # across descriptor dim

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = x.view(N, C, -1)
        
        # calculate residuals to each clusters
        residual = x_flatten.expand(self.num_clusters, -1, -1, -1).permute(1, 0, 2, 3) - \
            self.centroids.expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
        residual *= soft_assign.unsqueeze(2)
        vlad = residual.sum(dim=-1)

        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize

        return vlad


In [0]:
class EmbedNet(nn.Module):
    def __init__(self, base_model, net_vlad):
        super(EmbedNet, self).__init__()
        self.base_model = base_model
        self.net_vlad = net_vlad

    def forward(self, x):
        x = self.base_model(x)
        embedded_x = self.net_vlad(x)
        return embedded_x
      
class TripletNet(nn.Module):
    def __init__(self, embed_net):
        super(TripletNet, self).__init__()
        self.embed_net = embed_net

    def forward(self, a, p, n):
        embedded_a = self.embed_net(a)
        embedded_p = self.embed_net(p)
        embedded_n = self.embed_net(n)
        return embedded_a, embedded_p, embedded_n

    def feature_extract(self, x):
        return self.embed_net(x)

In [0]:
class HardTripletLoss(nn.Module):
    """Hard/Hardest Triplet Loss
    (pytorch implementation of https://omoindrot.github.io/triplet-loss)
    For each anchor, we get the hardest positive and hardest negative to form a triplet.
    """
    def __init__(self, margin=0.1, hardest=False, squared=False):
        """
        Args:
            margin: margin for triplet loss
            hardest: If true, loss is considered only hardest triplets.
            squared: If true, output is the pairwise squared euclidean distance matrix.
                If false, output is the pairwise euclidean distance matrix.
        """
        super(HardTripletLoss, self).__init__()
        self.margin = margin
        self.hardest = hardest
        self.squared = squared

    def forward(self, embeddings, labels):
        """
        Args:
            labels: labels of the batch, of size (batch_size,)
            embeddings: tensor of shape (batch_size, embed_dim)
        Returns:
            triplet_loss: scalar tensor containing the triplet loss
        """
        pairwise_dist = _pairwise_distance(embeddings, squared=self.squared)

        if self.hardest:
            # Get the hardest positive pairs
            mask_anchor_positive = _get_anchor_positive_triplet_mask(labels).float()
            valid_positive_dist = pairwise_dist * mask_anchor_positive
            hardest_positive_dist, _ = torch.max(valid_positive_dist, dim=1, keepdim=True)

            # Get the hardest negative pairs
            mask_anchor_negative = _get_anchor_negative_triplet_mask(labels).float()
            max_anchor_negative_dist, _ = torch.max(pairwise_dist, dim=1, keepdim=True)
            anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (
                    1.0 - mask_anchor_negative)
            hardest_negative_dist, _ = torch.min(anchor_negative_dist, dim=1, keepdim=True)

            # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
            triplet_loss = F.relu(hardest_positive_dist - hardest_negative_dist + 0.1)
            triplet_loss = torch.mean(triplet_loss)
        else:
            anc_pos_dist = pairwise_dist.unsqueeze(dim=2)
            anc_neg_dist = pairwise_dist.unsqueeze(dim=1)

            # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
            # triplet_loss[i, j, k] will contain the triplet loss of anc=i, pos=j, neg=k
            # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
            # and the 2nd (batch_size, 1, batch_size)
            loss = anc_pos_dist - anc_neg_dist + self.margin

            mask = _get_triplet_mask(labels).float()
            triplet_loss = loss * mask

            # Remove negative losses (i.e. the easy triplets)
            triplet_loss = F.relu(triplet_loss)

            # Count number of hard triplets (where triplet_loss > 0)
            hard_triplets = torch.gt(triplet_loss, 1e-16).float()
            num_hard_triplets = torch.sum(hard_triplets)

            triplet_loss = torch.sum(triplet_loss) / (num_hard_triplets + 1e-16)

        return triplet_loss


def _pairwise_distance(x, squared=False, eps=1e-16):
    # Compute the 2D matrix of distances between all the embeddings.

    cor_mat = torch.matmul(x, x.t())
    norm_mat = cor_mat.diag()
    distances = norm_mat.unsqueeze(1) - 2 * cor_mat + norm_mat.unsqueeze(0)
    distances = F.relu(distances)

    if not squared:
        mask = torch.eq(distances, 0.0).float()
        distances = distances + mask * eps
        distances = torch.sqrt(distances)
        distances = distances * (1.0 - mask)

    return distances


def _get_anchor_positive_triplet_mask(labels):
    # Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    indices_not_equal = torch.eye(labels.shape[0]).to(device).byte() ^ 1

    # Check if labels[i] == labels[j]
    labels_equal = torch.unsqueeze(labels, 0) == torch.unsqueeze(labels, 1)

    mask = indices_not_equal * labels_equal

    return mask


def _get_anchor_negative_triplet_mask(labels):
    # Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.

    # Check if labels[i] != labels[k]
    labels_equal = torch.unsqueeze(labels, 0) == torch.unsqueeze(labels, 1)
    mask = labels_equal ^ 1

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Check that i, j and k are distinct
    indices_not_same = torch.eye(labels.shape[0]).to(device).byte() ^ 1
    i_not_equal_j = torch.unsqueeze(indices_not_same, 2)
    i_not_equal_k = torch.unsqueeze(indices_not_same, 1)
    j_not_equal_k = torch.unsqueeze(indices_not_same, 0)
    distinct_indices = i_not_equal_j * i_not_equal_k * j_not_equal_k

    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = torch.eq(torch.unsqueeze(labels, 0), torch.unsqueeze(labels, 1))
    i_equal_j = torch.unsqueeze(label_equal, 2)
    i_equal_k = torch.unsqueeze(label_equal, 1)
    valid_labels = i_equal_j * (i_equal_k ^ 1)

    mask = distinct_indices * valid_labels   # Combine the two masks

    return mask

In [0]:
def initialize_model(model_name, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    base_model = None
    dim = 0

    if model_name == "resnet18":
        """ Resnet18
        """
        encoder = models.resnet18(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.conv1,
        encoder.bn1,
        encoder.relu,
        encoder.maxpool,
        encoder.layer1,
        encoder.layer2,
        encoder.layer3,
        encoder.layer4,
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        
    elif model_name == "resnet50":
        """ Resnet50
        """
        encoder = models.resnet50(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.conv1,
        encoder.bn1,
        encoder.relu,
        encoder.maxpool,
        encoder.layer1,
        encoder.layer2,
        encoder.layer3,
        encoder.layer4,
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
    elif model_name == "resnet101":
        """ Resnet101
        """
        encoder = models.resnet101(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.conv1,
        encoder.bn1,
        encoder.relu,
        encoder.maxpool,
        encoder.layer1,
        encoder.layer2,
        encoder.layer3,
        encoder.layer4,
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        
    elif model_name == "alexnet":
        """ alexnet
        """
        encoder = models.alexnet(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,
        
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        

    elif model_name == "vgg16":
        """ vgg16
        """
        encoder = models.vgg16(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,
        
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
    elif model_name == "vgg19":
        """ vgg19
        """
        encoder = models.vgg19(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,
        
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
  
    elif model_name == "densenet121":
        """ densenet121
        """
        encoder = models.densenet121(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,
        
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        
    elif model_name == "densenet161":
        """ densenet161
        """
        encoder = models.densenet161(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,
        
        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
        
    elif model_name == 'mobilenet':
        """ mobilenet_v2
        """
        encoder = models.mobilenet_v2(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,

        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)
    elif model_name == 'squeezenet':
        """ squeezenet
        """
        encoder = models.squeezenet.squeezenet1_1(pretrained=use_pretrained)
        base_model = nn.Sequential(
        encoder.features,

        )
        dim = list(base_model.parameters())[-1].shape[0]  # last channels (512)

    else:
        print("Invalid model name, exiting...")
        

    return base_model, dim

In [13]:
# model_list = ['resnet18','resnet50','resnet101','alexnet','vgg16','vgg19','densenet121','densenet161','mobilenet','squeezenet']
base_model_dict = {}
model_list = [model_name]
for i in model_list:
  base_model,dim =initialize_model(i,use_pretrained=True)
  print(i,'is completed , dim is ',dim)
  base_model_dict[i]=[base_model,dim]

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:02<00:00, 19875320.82it/s]


resnet18 is completed , dim is  512


In [0]:
model_name = model_name
base_model = base_model_dict[model_name][0]
dim = base_model_dict[model_name][1]

In [0]:
# Define model for embedding
net_vlad = NetVLAD(num_clusters=7, dim=dim, alpha=1.0)
model = EmbedNet(base_model, net_vlad).cuda()

In [0]:
# Define loss
criterion = HardTripletLoss(margin=0.1).cuda()

In [0]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [18]:
state_dict = torch.load(model_load_path)
model.load_state_dict(state_dict)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((128,128)),                    #이미지의 크기
#     torchvision.transforms.ColorJitter(hue=.05, saturation=.05), #사진의 밝기를 변화시키는 코드
#     torchvision.transforms.RandomHorizontalFlip(),               #죄우 대칭을 위한 코드
    torchvision.transforms.ToTensor()
    
])

query_imagenet_data = torchvision.datasets.ImageFolder(test_path, transform=transforms)
query_data_loader = torch.utils.data.DataLoader(query_imagenet_data,
                                          shuffle=True,
                                          batch_size=366,
                                          num_workers=0)


## total 5 classes : 81 images per class
refer_imagenet_data = torchvision.datasets.ImageFolder(test_path, transform=transforms)
refer_data_loader = torch.utils.data.DataLoader(refer_imagenet_data,
                                          shuffle=False,
                                          batch_size=366,
                                          num_workers=0)

## Reference Data(test):

{'AI': 14, 

 'Clock tower': 22,
 
 'Front door Child': 14,
 
 'Front door Sejong': 24,
 
 'Stone statue': 7}
 
 ## Reference Data(train):

{'AI': 101, 

 'Clock tower': 41,
 
 'Front door Child': 83,
 
 'Front door Sejong': 46,
 
 'Stone statue': 95}

In [0]:
refer_np =None
refer_label = None
query_np = None
query_label = None

for query_image,query_label in query_data_loader :
  query_output = model(query_image.cuda()) 
  X_query = query_output.cpu().data.numpy()
  query_label = query_label.cpu().data.numpy()
  query_np = query_image.cpu().data.numpy()

for refer_image,refer_label in refer_data_loader :
  refer_output = model(refer_image.cuda()) 
  X_refer = refer_output.cpu().data.numpy()
  refer_label = refer_label.cpu().data.numpy()
  refer_np = refer_image.cpu().data.numpy()

In [0]:
# query_img = np.transpose(query_np, (0, 2, 3, 1))
# plt.figure(figsize=(5,5))
# plt.axis('off')
# plt.imshow(query_img[select_idx])

In [0]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=81, algorithm ='ball_tree').fit(X_refer) # X_refer shape (batch size , dimension)
distances, indices = nbrs.kneighbors(X_query)   # X_query shape (batch_size, dimension)

In [23]:
print(X_query.shape)  # query vector shape : batch size, dimension
print(query_np.shape)  # query numpy shape : batch size , channel , height, width
print(query_label.shape)   # query label shape : batch size[class idx] 

(81, 3584)
(81, 3, 128, 128)
(81,)


In [0]:
count_to_class ={}
for i in refer_label:
  if not i in count_to_class:
    count_to_class[i]=1
  else:
    count_to_class[i]+=1

In [25]:
count_to_class

{0: 14, 1: 22, 2: 14, 3: 24, 4: 7}

In [0]:
# y_true : label(interger), y_pred : label(list)
def get_TFtable(y_true,y_pred):
  table = []
  TP =[]
  FP = []
  for i in y_pred:
    if i==y_true:
      table.append('true')
    else:
      table.append('false')
  return table

def evaluateAP(y_true, y_pred):
  N_R = len(y_pred) 
#   print('image are retrieved N : {}'.format(N_R))
  Rank_1 = None
  P_5 = None
  P_10 = None
  RP_half = None
  
  s1 = 0
  s2 = 0
  s3 = 0 
  s4 = 0
  
  tf_table=get_TFtable(y_true,y_pred)
  reverse_table = tf_table.copy()
  reverse_table.reverse()
  index_T=len(reverse_table)-reverse_table.index('true')-1 #table의 마지막 TP index
  K = count_to_class[y_true]
  count_TP =0
  sum_precision = 0
  recall = 0
  precision = 0
  for i in range(0,index_T+1):
    if precision>=0.5 and RP_half == None:
      RP_half = recall

#       print('recall at precision 0.5 : {}'.format(RP_half))
    if i==6:
        P_5 = precision
#         print('precision{}@5 : {}'.format(i,P_5))
#         print('R_{} precision : {}'.format(i,P_20))
    if i==11:
        P_10 = precision
#         print('precision{}@10 : {}'.format(i,P_10))
#         print('R_{} precision : {}'.format(i,P_50))
    if tf_table[i]=='true':
      if Rank_1 == None and i>=1:
        Rank_1 = i

#         print('Rank_1 index : {} '.format(Rank_1))
        
      count_TP+=1
      recall = count_TP/index_T
      precision = count_TP/(i+1)
      sum_precision += precision

#       print('t_i : {} , recall : {} , precision : {} '.format(i,recall,precision))
  print('Average Precision query {} : {} '.format(y_true,sum_precision / K))
  return sum_precision /K,Rank_1,P_5,RP_half


In [27]:
import itertools

sum_to_class ={}
map_to_class = {}
sumRank1_to_class ={}
sumP5_to_class ={}
sumP10_to_class ={}
sumRPhalf_to_class ={}
map2_to_class = {}
map3_to_class = {}
map5_to_class = {}


total_MAP = 0
sum_AP = 0
for i in range(0,len(query_label)):
  keys = np.ndarray.tolist(indices[i]) 
  values = np.ndarray.tolist(distances[i]) 

  find_dict = dict(zip(keys,values)) 

  similar = sorted(find_dict.items(), key=lambda find_dict: values)[:81]
  similar_idx = [i[0] for i in similar]

  y_true = query_label[i]
#   print('query label : ',y_true)
  
  y_pred = [refer_label[i] for i in similar_idx]
#   print('retrieval label : ',y_pred)
  
  AP,s1,s2,s4=evaluateAP(y_true,y_pred)
#   print(s1,s2,s4)
  sum_AP+=AP
  if not y_true in sum_to_class.keys():
    sum_to_class[y_true]=AP
  else:
    sum_to_class[y_true]+=AP
  if not y_true in sumRank1_to_class.keys():
    sumRank1_to_class[y_true]=s1
  else:
    sumRank1_to_class[y_true]+=s1
  if not y_true in sumP5_to_class.keys():
    sumP5_to_class[y_true]=s2
  else:
#     print(s2)
    sumP5_to_class[y_true]+=s2

  if not y_true in sumRPhalf_to_class.keys():
    sumRPhalf_to_class[y_true]=s4
  else:
    sumRPhalf_to_class[y_true]+=s4

    
total_MAP = sum_AP/len(query_label)
for key,value in sum_to_class.items():
  map_to_class[key]=sum_to_class[key]/count_to_class[key]

for key,value in sumRank1_to_class.items():
  map2_to_class[key]=sumRank1_to_class[key]/count_to_class[key]
  
for key,value in sumP5_to_class.items():
  map3_to_class[key]=sumP5_to_class[key]/count_to_class[key]
  
for key,value in sumRPhalf_to_class.items():
  map5_to_class[key]=sumRPhalf_to_class[key]/count_to_class[key]


Average Precision query 2 : 0.5297665909705079 
Average Precision query 3 : 0.7498153074413572 
Average Precision query 1 : 0.9791375291375292 
Average Precision query 1 : 0.9850756439961837 
Average Precision query 1 : 0.29136847106520153 
Average Precision query 0 : 1.0 
Average Precision query 3 : 0.6812768404695922 
Average Precision query 1 : 0.6546101636975652 
Average Precision query 0 : 1.0 
Average Precision query 2 : 0.6225112031230149 
Average Precision query 1 : 0.40468785973420457 
Average Precision query 2 : 0.4244018748677134 
Average Precision query 1 : 0.9815184815184814 
Average Precision query 2 : 0.41238430513640595 
Average Precision query 2 : 0.630455342317154 
Average Precision query 2 : 0.3405263750548034 
Average Precision query 1 : 0.9517205208584518 
Average Precision query 1 : 0.9380474917239624 
Average Precision query 3 : 0.6703621593220598 
Average Precision query 1 : 0.9848484848484849 
Average Precision query 0 : 1.0 
Average Precision query 4 : 0.82602

In [28]:
hist_dict={}
hist_list =[]
for name, idx in refer_imagenet_data.class_to_idx.items():
  hist_dict = {'base_model':model_name,'epochs':epochs,'batch_size':batch_size,'class_name':name,'map_to_class':map_to_class[idx],'count_to_class':count_to_class[idx],'Rank1_to_class':map2_to_class[idx],'P5_to_class':map3_to_class[idx],'RP_0.5':map5_to_class[idx]}
  hist_list.append(hist_dict)
  print('base model name : {} , epoch : {}, batch size : {} , class name : {} , Mean Average Precision to class : {} , query counts to class : {}, {}, {} ,{} '.format(hist_dict['base_model'],hist_dict['epochs'] ,hist_dict['batch_size'],hist_dict['class_name'],hist_dict['map_to_class'],hist_dict['count_to_class'],hist_dict['Rank1_to_class'],hist_dict['P5_to_class'],hist_dict['RP_0.5']))

base model name : resnet18 , epoch : 50, batch size : 256 , class name : AI , Mean Average Precision to class : 1.0 , query counts to class : 14, 1.0, 1.0 ,0.0769230769230769 
base model name : resnet18 , epoch : 50, batch size : 256 , class name : Clock tower , Mean Average Precision to class : 0.8813520378028258 , query counts to class : 22, 1.6818181818181819, 1.0 ,0.031401239593915505 
base model name : resnet18 , epoch : 50, batch size : 256 , class name : Front door Child , Mean Average Precision to class : 0.5251225266048548 , query counts to class : 14, 3.0, 0.7452380952380953 ,0.026315798637227206 
base model name : resnet18 , epoch : 50, batch size : 256 , class name : Front door Sejong , Mean Average Precision to class : 0.6583037381761169 , query counts to class : 24, 1.4166666666666667, 0.796527777777778 ,0.023565038056725423 
base model name : resnet18 , epoch : 50, batch size : 256 , class name : Stone statue , Mean Average Precision to class : 0.9598396501457724 , query

In [0]:
# fig=plt.figure(figsize=(16, 16))
# columns = 5
# rows = 20
# ax = []

# idx = 0
# for i in range(rows):
#   for j in similar_idx:
#     idx+=1
#     ax.append( fig.add_subplot(rows, columns, idx) )
#     ax[-1].set_title("top :"+str(idx))  # set title
#     plt.imshow(refer_img[j])

# plt.show()

In [0]:
import csv

with open(history_path, 'a') as f:  # Just use 'w' mode in 3.x
  for i in range(0,len(hist_list)):
    w = csv.DictWriter(f, hist_list[i].keys())
    if i==0:
      w.writeheader()
    w.writerow(hist_list[i])
  f.close()
